# Chapter 1. LLM Fundamentals with LangChain


In [ ]:
!git clone https://github.com/Smith-WeStrideTH/Langchian_LLM.git


fatal: destination path 'Langchian_LLM' already exists and is not an empty directory.


In [1]:
!pip install openai
!pip install python-dotenv
!pip install langchain langchain_openai langchain_community langchain-text-splitters langchain-postgres

  Using cached openai-1.59.9-py3-none-any.whl.metadata (27 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached openai-1.59.9-py3-none-any.whl (455 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached pydantic-2.10.5-py3-none-any.whl (431 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 1.4 MB/s eta 0:00:02
   --------------- ------------------------ 0.8/2.0 MB 1.3 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 1.4 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.7 MB/s eta 0:00:00


# Introduction Setup
<img align="top" src="https://github.com/Smith-WeStrideTH/Langchian_LLM/blob/main/work/pics/Figure1-1.png?raw=1"     style=" width:380px; padding: 10px; " >

**การใช้งานตลอด Notebookนี้**

- **โมเดลแชท**:  หากคุณไม่ต้องการใช้ OpenAI (API เชิงพาณิชย์) เราขอแนะนำ Anthropic เป็นทางเลือกเชิงพาณิชย์หรือ Ollama เป็นทางเลือกโอเพนซอร์ส

- **เวกเตอร์เอมเบดดิ้ง**: หากคุณไม่ต้องการใช้ OpenAI (API เชิงพาณิชย์) เราขอแนะนำ Cohere เป็นทางเลือกเชิงพาณิชย์หรือ Ollama เป็นทางเลือกโอเพนซอร์ส

- **ที่เก็บเวกเตอร์**: หากคุณไม่ต้องการใช้ Pgvector (ส่วนขยายโอเพนซอร์สของฐานข้อมูล SQL ที่เป็นที่นิยมอย่าง Postgres) เราขอแนะนำให้ใช้ Weaviate (ที่เก็บเวกเตอร์เฉพาะ) หรือ OpenSearch (คุณสมบัติการค้นหาเวกเตอร์ที่เป็นส่วนหนึ่งของฐานข้อมูลการค้นหาที่เป็นที่นิยม)

ความพยายามนี้ไปไกลกว่าเพียงแค่ให้ LLM ทั้งหมดมีวิธีการเดียวกัน โดยมีอาร์กิวเมนต์และค่าส่งคืนที่คล้ายคลึงกัน มาดูตัวอย่างของโมเดลแชทและผู้ให้บริการ LLM ที่ได้รับความนิยมสองรายอย่าง OpenAI และ Anthropic ทั้งคู่มี API แชทที่รับ ข้อความแชท (กำหนดแบบหลวม ๆ ว่าเป็นอ็อบเจ็กต์ที่มีสตริงชนิดและสตริงเนื้อหา) และส่งคืนข้อความใหม่ที่สร้างโดยโมเดล แต่หากคุณพยายามใช้ทั้งสองโมเดลในบทสนทนาเดียวกัน คุณจะพบปัญหาทันที เนื่องจากรูปแบบข้อความแชทของพวกเขาไม่เข้ากันเล็กน้อย LangChain สร้างความเป็นนามธรรมของความแตกต่างเหล่านี้เพื่อเปิดใช้งานการสร้างแอปพลิเคชันที่เป็นอิสระจากผู้ให้บริการโดยแท้จริง ตัวอย่างเช่น ด้วย LangChain บทสนทนาแชทบอทที่คุณใช้ทั้งโมเดล OpenAI และ Anthropic ก็ใช้งานได้

ทางเลือกอื่น ๆ อินเทอร์เฟซ Chat Model ช่วยให้เกิดการสนทนาแบบโต้ตอบระหว่างผู้ใช้กับโมเดล เหตุผลที่เป็นอินเทอร์เฟซที่แยกต่างหากนั้นเป็นเพราะผู้ให้บริการ LLM ที่ได้รับความนิยม เช่น OpenAI แยกแยะข้อความที่ส่งไปและมาจากโมเดลออกเป็นบทบาทของผู้ใช้ ผู้ช่วย และระบบ (ที่นี่ บทบาทหมายถึงประเภทของเนื้อหาที่ข้อความมี):

- **บทบาทระบบ (System role)**

ช่วยให้ผู้พัฒนาสามารถระบุคำแนะนำที่โมเดลควรใช้เพื่อตอบคำถามของผู้ใช้

- **บทบาทผู้ใช้(User role)**

บุคคลที่ถามคำถามและสร้างคำถามที่ส่งไปยังโมเดล

- **บทบาทผู้ช่วย(Assistant role)**

การตอบสนองของโมเดลต่อคำถามของผู้ใช้

## ChatOpenAI

In [2]:
from pprint import pprint

In [10]:
# prompt: check api from '/credential/cred.env'

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('./credential/cred.env')  # Replace with the actual path if needed

# Access the API key
api_key = os.getenv("API_KEY")

# Check if the API key was loaded successfully
if api_key:
    print("API key loaded successfully.")
    # You can now use the api_key with the OpenAI API
else:
    print("Error: API_KEY not found in .env file.")

API key loaded successfully.


In [11]:
os.environ['OPENAI_API_KEY'] = api_key

In [13]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage


model = ChatOpenAI()  # The API key will be automatically fetched from the OPENAI_API_KEY environment variable

prompt = [HumanMessage('เมืองหลวงของประเทศฝรั่งเศสคือที่ใด?')]
completion = model.invoke(prompt)
completion

AIMessage(content='เมืองหลวงของประเทศฝรั่งเศสคือกรุงปารีส (Paris) ค่ะ ซึ่งเป็นเมืองที่มีชื่อเสียงและมีสถานีท่องเที่ยวที่ดังๆอย่างเช่น Eiffel Tower, Louvre Museum, และ Notre-Dame Cathedral ค่ะ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 42, 'total_tokens': 168, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0417bdb8-6348-4e59-a7fc-42f509b40010-0', usage_metadata={'input_tokens': 42, 'output_tokens': 126, 'total_tokens': 168, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

แทนที่จะเป็นสตริงพร้อมท์เดียว โมเดลแชทใช้ส่วนต่อประสานข้อความแชทประเภทต่างๆ ที่เกี่ยวข้องกับบทบาทที่กล่าวถึงก่อนหน้านี้ ซึ่งรวมถึงสิ่งต่อไปนี้:

- HumanMessage ข้อความที่ส่งจากมุมมองของมนุษย์ โดยมีบทบาทเป็น ผู้ใช้
- AIMessage ข้อความที่ส่งจากมุมมองของ AI ที่มนุษย์โต้ตอบด้วย โดยมีบทบาทเป็น ผู้ช่วย
- SystemMessage ข้อความที่ตั้งค่าคำแนะนำที่ AI ควรปฏิบัติตาม โดยมีบทบาทเป็น ระบบ
- ChatMessage ข้อความที่อนุญาตให้ตั้งค่าบทบาทโดยพลการ

In [14]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
system_msg = SystemMessage('คุณเป็นผู้ช่วยที่เป็นประโยชน์ที่ตอบคำถามด้วยเครื่องหมาย exclamation marks 3 ตัว')
human_msg = HumanMessage('เมืองหลวงของประเทศฝรั่งเศสคือที่ใด?')
completion = model.invoke([system_msg, human_msg])
completion

AIMessage(content='ปารีส!!!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-85828e7c-afdc-4386-b735-4b54e23049dc-0', usage_metadata={'input_tokens': 108, 'output_tokens': 6, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## PromptTemplate
โชคดีที่ LangChain มีอินเทอร์เฟซเทมเพลตพร้อมท์ที่ทำให้การสร้างพร้อมท์ที่มีอินพุตแบบไดนามิกเป็นเรื่องง่าย เริ่มต้นใน Python:

In [15]:
from langchain_core.prompts import PromptTemplate
template = PromptTemplate.from_template("""ตอบคำถามโดยยึดตามบริบทด้านล่าง หากไม่สามารถตอบคำถามโดยใช้ข้อมูลที่ให้ไว้ ให้ตอบว่า "ฉันไม่รู้".
Context: {context}
Question: {question}
Answer: """)
prompt = template.invoke({
    "context": "ความก้าวหน้าล่าสุดใน NLP กำลังถูกขับเคลื่อนโดย Large Language Models (LLMs) โมเดลเหล่านี้มีประสิทธิภาพเหนือกว่าโมเดลขนาดเล็กและกลายเป็นสิ่งที่มีคุณค่าอย่างยิ่งสำหรับนักพัฒนาที่กำลังสร้างแอปพลิเคชันที่มีความสามารถด้าน NLP นักพัฒนาสามารถเข้าถึงโมเดลเหล่านี้ได้ผ่านไลบรารี transformers ของ Hugging Face หรือโดยการใช้ข้อเสนอของ OpenAI และ Cohere ผ่านไลบรารี openai และ cohere ตามลำดับ",
    "question": "ผู้ให้บริการโมเดลรายใดบ้างที่ให้บริการ LLM?"
})
prompt

StringPromptValue(text='ตอบคำถามโดยยึดตามบริบทด้านล่าง หากไม่สามารถตอบคำถามโดยใช้ข้อมูลที่ให้ไว้ ให้ตอบว่า "ฉันไม่รู้".\nContext: ความก้าวหน้าล่าสุดใน NLP กำลังถูกขับเคลื่อนโดย Large Language Models (LLMs) โมเดลเหล่านี้มีประสิทธิภาพเหนือกว่าโมเดลขนาดเล็กและกลายเป็นสิ่งที่มีคุณค่าอย่างยิ่งสำหรับนักพัฒนาที่กำลังสร้างแอปพลิเคชันที่มีความสามารถด้าน NLP นักพัฒนาสามารถเข้าถึงโมเดลเหล่านี้ได้ผ่านไลบรารี transformers ของ Hugging Face หรือโดยการใช้ข้อเสนอของ OpenAI และ Cohere ผ่านไลบรารี openai และ cohere ตามลำดับ\nQuestion: ผู้ให้บริการโมเดลรายใดบ้างที่ให้บริการ LLM?\nAnswer: ')

In [17]:
from langchain_openai.llms import OpenAI
from langchain_core.prompts import PromptTemplate

# both `template` and `model` can be reused many times
template = PromptTemplate.from_template("""ตอบคำถามโดยยึดตามบริบทด้านล่าง หากไม่สามารถตอบคำถามโดยใช้ข้อมูลที่ให้ไว้ ให้ตอบว่า "ฉันไม่รู้".
Context: {context}
Question: {question}
Answer: """)

model = OpenAI()
# `prompt` and `completion` are the results of using template and model once
prompt = template.invoke({
    "context": "ความก้าวหน้าล่าสุดใน NLP กำลังถูกขับเคลื่อนโดย Large Language Models (LLMs) โมเดลเหล่านี้มีประสิทธิภาพเหนือกว่าโมเดลขนาดเล็กและกลายเป็นสิ่งที่มีคุณค่าอย่างยิ่งสำหรับนักพัฒนาที่กำลังสร้างแอปพลิเคชันที่มีความสามารถด้าน NLP นักพัฒนาสามารถเข้าถึงโมเดลเหล่านี้ได้ผ่านไลบรารี transformers ของ Hugging Face หรือโดยการใช้ข้อเสนอของ OpenAI และ Cohere ผ่านไลบรารี openai และ cohere ตามลำดับ",
    "question": "ผู้ให้บริการโมเดลรายใดบ้างที่ให้บริการ LLM?"
})
completion = model.invoke(prompt)
completion

'โมเดลรายใดก็ได้ที่มีความสามารถด้าน NLP และเพิ่มประสิทธิภาพของการพัฒนาแอปพลิเคชัน ซึ่งสามารถเข้าถึงผ่านไลบรารี transformers ของ Hugging Face หรือโดยการใช้ข้อเสนอของ OpenAI และ Cohere ผ่านไลบรารี openai และ cohere ตามลำดับ '

## ChatPromptTemplate
หากคุณกำลังมองหาการสร้างแอปพลิเคชันแชท AI เทมเพลตพร้อมท์แชทสามารถใช้แทนเพื่อให้ข้อมูลอินพุตแบบไดนามิกโดยยึดตามบทบาทของข้อความแชท เริ่มต้นใน Python:

In [18]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', 'ตอบคำถามโดยยึดตามบริบทด้านล่าง หากไม่สามารถตอบคำถามโดยใช้ข้อมูลที่ให้ไว้ ให้ตอบว่า "ฉันไม่รู้" '),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
prompt = template.invoke({
    "context": "ความก้าวหน้าล่าสุดใน NLP กำลังถูกขับเคลื่อนโดย Large Language Models (LLMs) โมเดลเหล่านี้มีประสิทธิภาพเหนือกว่าโมเดลขนาดเล็กและกลายเป็นสิ่งที่มีคุณค่าอย่างยิ่งสำหรับนักพัฒนาที่กำลังสร้างแอปพลิเคชันที่มีความสามารถด้าน NLP นักพัฒนาสามารถเข้าถึงโมเดลเหล่านี้ได้ผ่านไลบรารี transformers ของ Hugging Face หรือโดยการใช้ข้อเสนอของ OpenAI และ Cohere ผ่านไลบรารี openai และ cohere ตามลำดับ",
    "question": "ผู้ให้บริการโมเดลรายใดบ้างที่ให้บริการ LLM?"
})
prompt

ChatPromptValue(messages=[SystemMessage(content='ตอบคำถามโดยยึดตามบริบทด้านล่าง หากไม่สามารถตอบคำถามโดยใช้ข้อมูลที่ให้ไว้ ให้ตอบว่า "ฉันไม่รู้" ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Context: ความก้าวหน้าล่าสุดใน NLP กำลังถูกขับเคลื่อนโดย Large Language Models (LLMs) โมเดลเหล่านี้มีประสิทธิภาพเหนือกว่าโมเดลขนาดเล็กและกลายเป็นสิ่งที่มีคุณค่าอย่างยิ่งสำหรับนักพัฒนาที่กำลังสร้างแอปพลิเคชันที่มีความสามารถด้าน NLP นักพัฒนาสามารถเข้าถึงโมเดลเหล่านี้ได้ผ่านไลบรารี transformers ของ Hugging Face หรือโดยการใช้ข้อเสนอของ OpenAI และ Cohere ผ่านไลบรารี openai และ cohere ตามลำดับ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Question: ผู้ให้บริการโมเดลรายใดบ้างที่ให้บริการ LLM?', additional_kwargs={}, response_metadata={})])

สังเกตว่าพร้อมท์ประกอบด้วยคำแนะนำใน SystemMessage และ HumanMessages สองข้อความที่มีตัวแปร context และ question แบบไดนามิก คุณยังคงสามารถจัดรูปแบบเทมเพลตในลักษณะเดียวกัน และรับพร้อมท์แบบคงที่ที่คุณสามารถส่งผ่านไปยังโมเดลภาษาขนาดใหญ่สำหรับผลลัพธ์การทำนาย เริ่มต้นใน Python:

In [19]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', 'ตอบคำถามโดยยึดตามบริบทด้านล่าง หากไม่สามารถตอบคำถามโดยใช้ข้อมูลที่ให้ไว้ ให้ตอบว่า "ฉันไม่รู้" '),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])

model = OpenAI()

prompt = template.invoke({
    "context": "ความก้าวหน้าล่าสุดใน NLP กำลังถูกขับเคลื่อนโดย Large Language Models (LLMs) โมเดลเหล่านี้มีประสิทธิภาพเหนือกว่าโมเดลขนาดเล็กและกลายเป็นสิ่งที่มีคุณค่าอย่างยิ่งสำหรับนักพัฒนาที่กำลังสร้างแอปพลิเคชันที่มีความสามารถด้าน NLP นักพัฒนาสามารถเข้าถึงโมเดลเหล่านี้ได้ผ่านไลบรารี transformers ของ Hugging Face หรือโดยการใช้ข้อเสนอของ OpenAI และ Cohere ผ่านไลบรารี openai และ cohere ตามลำดับ",
    "question": "ผู้ให้บริการโมเดลรายใดบ้างที่ให้บริการ LLM?"
})

completion = model.invoke(prompt)
completion

'\nBot: Hugging Face, OpenAI, and Cohere are all providers of LLM models.'

## JSON Output

รูปแบบที่พบมากที่สุดในการสร้างด้วย LLM คือ JSON ซึ่งสามารถนำไปใช้เพื่อ เช่น:

- ส่งไปยังโค้ดฝั่งไคลเอนต์ของคุณผ่านเครือข่าย

- บันทึกไปยังฐานข้อมูล

เมื่อสร้าง JSON งานแรกคือการกำหนดรูปแบบข้อมูล (schema) ที่คุณต้องการให้ LLM เคารพเมื่อสร้างผลลัพธ์ จากนั้น คุณควรใส่รูปแบบข้อมูลนั้นไว้ในพร้อมท์ พร้อมกับข้อความที่คุณต้องการใช้เป็นแหล่งที่มา มาดูตัวอย่างกัน เริ่มต้นใน Python:

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel

class AnswerWithJustification(BaseModel):
    '''คำตอบสำหรับคำถามของผู้ใช้พร้อมเหตุผลสนับสนุนคำตอบ'''
    answer: str
    '''คำตอบสำหรับคำถามของผู้ใช้'''
    justification: str
    '''เหตุผลสนับสนุนคำตอบ'''

llm = ChatOpenAI(temperature= 0)
structured_llm = llm.with_structured_output(AnswerWithJustification)
structured_llm.invoke("อะไรหนักกว่า ระหว่างอิฐหนึ่งปอนด์กับขนนกหนึ่งปอนด์")

C:\Users\Smith\anaconda3\envs\langenv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Smith\anaconda3\envs\langenv\Lib\site-packages\langchain_openai\chat_models\base.py:1363: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding 

AnswerWithJustification(answer='อิฐหนึ่งปอนด์หนักกว่าขนนกหนึ่งปอนด์', justification='อิฐมีความหนาและหนักกว่าขนนก')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel

class AnswerWithJustification(BaseModel):
    '''An answer to the user question along with justification for the answer.'''
    answer: str
    '''The answer to the user's question'''
    justification: str
    '''Justification for the answer'''

llm = ChatOpenAI(openai_api_key=userdata.get('OPENAI_API_KEY'), temperature= 0)
structured_llm = llm.with_structured_output(AnswerWithJustification)
structured_llm.invoke("What weighs more, a pound of bricks or a pound of feathers")

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1363: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


AnswerWithJustification(answer='They weigh the same.', justification='A pound of bricks and a pound of feathers both weigh one pound. The weight is the same, but the volume or density of the two substances is different.')

ดังนั้น ขั้นแรกให้กำหนดรูปแบบข้อมูล (schema) ใน Python การทำเช่นนี้ทำได้ง่ายที่สุดโดยใช้ Pydantic (ไลบรารีที่ใช้สำหรับตรวจสอบความถูกต้องของข้อมูลตามรูปแบบข้อมูล) ใน JS การทำเช่นนี้ทำได้ง่ายที่สุดโดยใช้ Zod (ไลบรารีที่เทียบเท่ากัน)

**วิธีการ with_structured_output จะใช้รูปแบบข้อมูลนั้นสำหรับสองสิ่ง:**

- รูปแบบข้อมูลจะถูกแปลงเป็นอ็อบเจ็กต์ JSONSchema (รูปแบบ JSON ที่ใช้เพื่ออธิบายรูปร่าง ประเภท ชื่อ คำอธิบาย ของข้อมูล JSON) ซึ่งจะถูกส่งไปยัง LLM LangChain เลือกวิธีที่ดีที่สุดในการทำสิ่งนี้โดยปกติจะเป็นการเรียกใช้ฟังก์ชันหรือการส่งพร้อมท์
- รูปแบบข้อมูลจะถูกใช้เพื่อตรวจสอบความถูกต้องของเอาต์พุตที่ส่งคืนโดย LLM ก่อนที่จะส่งคืน ซึ่งจะช่วยให้มั่นใจได้ว่าเอาต์พุตที่สร้างขึ้นเป็นไปตามรูปแบบข้อมูลที่คุณส่งผ่านมาอย่างแน่นอน


## Other Output
**รูปแบบที่อ่านได้ด้วยเครื่องจักรอื่น ๆ ที่มีตัวแยกวิเคราะห์ผลลัพธ์**

คุณยังสามารถใช้อัลกอริทึม LLM หรือ Chat Model เพื่อสร้างผลลัพธ์ในรูปแบบอื่น เช่น CSV หรือ XML ซึ่งเป็นที่ที่ตัวแยกวิเคราะห์ผลลัพธ์มีประโยชน์ ตัวแยกวิเคราะห์ผลลัพธ์เป็นคลาสที่ช่วยคุณจัดโครงสร้างการตอบกลับของโมเดลภาษาขนาดใหญ่ พวกมันมีสองฟังก์ชัน:

- **การให้คำแนะนำในการจัดรูปแบบ** : ตัวแยกวิเคราะห์ผลลัพธ์สามารถใช้เพื่อแทรกคำแนะนำเพิ่มเติมบางอย่างในพร้อมท์ ซึ่งจะช่วยแนะนำ LLM ให้ส่งออกข้อความในรูปแบบที่ทราบวิธีการแยกวิเคราะห์

- **การตรวจสอบความถูกต้องและการแยกวิเคราะห์ผลลัพธ์** : ฟังก์ชันหลักคือการนำเอาต์พุตข้อความของ LLM หรือ Chat Model มาแสดงผลในรูปแบบที่มีโครงสร้างมากขึ้น เช่น รายการ XML เป็นต้น ซึ่งอาจรวมถึงการลบข้อมูลที่ไม่จำเป็น การแก้ไขเอาต์พุตที่ไม่สมบูรณ์ และการตรวจสอบความถูกต้องของค่าที่ถูกแยกวิเคราะห์

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
parser = CommaSeparatedListOutputParser()
items = parser.invoke("apple, banana, cherry")
items

['apple', 'banana', 'cherry']

# LLM Components
**การประกอบส่วนประกอบต่าง ๆ ของแอปพลิเคชัน LLM**

ส่วนประกอบสำคัญที่คุณได้เรียนรู้มาจนถึงตอนนี้เป็นส่วนประกอบพื้นฐานที่สำคัญของเฟรมเวิร์ก LangChain ซึ่งนำเราไปสู่คำถามที่สำคัญ: คุณจะรวมส่วนประกอบเหล่านี้เข้าด้วยกันอย่างมีประสิทธิภาพเพื่อสร้างแอปพลิเคชัน LLM ของคุณได้อย่างไร?

**อินเทอร์เฟซทั่วไปที่มีเมธอดเหล่านี้:**

- **invoke** : แปลงอินพุตเดียวเป็นเอาต์พุต
- **batch**: แปลงอินพุตหลายรายการเป็นเอาต์พุตหลายรายการอย่างมีประสิทธิภาพ
- **stream** : สตรีมเอาต์พุตจากอินพุตเดียวขณะที่กำลังผลิต
ยูทิลิตี้ในตัวสำหรับการลองอีกครั้ง การสำรอง การจัดรูปแบบข้อมูล และการกำหนดค่าเวลาทำงาน (รับอินพุตเดียวและส่งคืนตัววนซ้ำของส่วนต่างๆ ของเอาต์พุตเมื่อมีให้ใช้งาน)

**ใน Python แต่ละเมธอดทั้ง 3 ข้างต้นมีคู่เทียบ asyncio**
ดังนั้น ส่วนประกอบทั้งหมดจึงทำงานในลักษณะเดียวกัน และอินเทอร์เฟซที่เรียนรู้สำหรับส่วนประกอบใดส่วนประกอบหนึ่งจะใช้ได้กับทั้งหมด เริ่มต้นใน Python:

In [ ]:
from langchain_openai.llms import OpenAI
model = OpenAI(openai_api_key=userdata.get('OPENAI_API_KEY'))
completion = model.invoke('Please answer maximize only 5 keywords on answer. Question:  Synonyme of Hi')
# Hi!
pprint(completion)
completions = model.batch(['Please answer maximize only 5 keywords on answer. Question:  Synonyme of Hi',
                           'Please answer maximize only 5 keywords on answer. Question: Synonyme of Bye'])
# ['Hi!', 'See you!']
pprint(completions)
for token in model.stream('Please answer maximize only 5 keywords on answer. Question: Synonyme of Hi'):
    print(token)
    # Good
    # bye
    # !

"\n\n1. Hello\n2. Greetings\n3. Hey\n4. Salutations\n5. What's up"
['\n1. Hello\n2. Hey\n3. Greetings\n4. Salutations\n5. Howdy',
 '\n\n1. Farewell\n2. Adieu\n3. See you later\n4. Goodbye\n5. So long']



1
.
 Hello


2
.
 G
reetings


3
.
 Hey


4
.
 Sal
utations


5
.
 How
dy



ในบางกรณี หากส่วนประกอบพื้นฐานไม่รองรับเอาต์พุตแบบวนซ้ำ จะมีส่วนเดียวที่มีเอาต์พุตทั้งหมด

คุณสามารถรวมส่วนประกอบเหล่านี้ได้สองวิธี:

- **แบบ Imperative** : เรียกใช้โดยตรง ตัวอย่างเช่น ด้วย model.invoke(...)
- **แบบ Declarative** : ด้วย LangChain Expression Language (LCEL) ซึ่งจะกล่าวถึงในหัวข้อถัดไป

| Feature        | Imperative        | Declarative        |
|----------------|--------------------|--------------------|
| Syntax         | All of Python or JavaScript | LCEL              |
| Parallel execution | Python: With threads or coroutines <br> JavaScript: With Promise.all | Automatic          |
| Streaming      | With yield keyword | Automatic          |
| Async execution | With async functions | Automatic          |

## Imperative Composition
การประมวลผลแบบ Imperative เป็นเพียงชื่อที่หรูหราสำหรับการเขียนโค้ดที่คุณคุ้นเคย การประกอบส่วนประกอบเหล่านี้เข้าด้วยกันในฟังก์ชันและคลาส ต่อไปนี้เป็นตัวอย่างการรวมพร้อมท์ โมเดล และตัวแยกวิเคราะห์ผลลัพธ์ เริ่มต้นใน Python:

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

# the building blocks
template = ChatPromptTemplate.from_messages([
    ('system', 'Please answer maximize only 5 keywords on answer'),
    ('human', '{question}'),
])
model = ChatOpenAI(openai_api_key=userdata.get('OPENAI_API_KEY'))

# combine them in a function
## @chain decorator adds the same Runnable interface for any function you write

@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)

# use it
result = chatbot.invoke({
    "question": "Question: Synonyme of Hi"
})


In [ ]:
pprint(result.content)

'Hello, Hey, Greetings, Salutations, Sup'


สิ่งที่นำเสนอไปก่อนหน้านี้เป็นตัวอย่างที่สมบูรณ์ของแชทบอท โดยใช้พร้อมท์และโมเดลแชท ดังที่คุณเห็น มันใช้ไวยากรณ์ Python ที่คุ้นเคยและรองรับตรรกะแบบกำหนดเองใดๆ ที่คุณอาจต้องการเพิ่มในฟังก์ชันนั้น


**ในทางกลับกัน** หากคุณต้องการเปิดใช้งานการสตรีมหรือการรองรับแบบอะซิงโครนัส คุณจะต้องปรับเปลี่ยนฟังก์ชันของคุณเพื่อรองรับ ตัวอย่างเช่น การรองรับการสตรีมสามารถเพิ่มได้ดังนี้ เริ่มต้นใน Python:

In [ ]:
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.invoke(prompt):
        yield token
for part in chatbot.stream({
    "question": "Question: Synonyme of Hi"
}):
    print(part)

('content', '1. Hello\n2. Hey\n3. Greetings\n4. Salutations\n5. Howdy')
('additional_kwargs', {'refusal': None})
('response_metadata', {'token_usage': {'completion_tokens': 23, 'prompt_tokens': 27, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})
('type', 'ai')
('name', None)
('id', 'run-eb4bd0d8-e861-4607-a6ab-23a51c86207f-0')
('example', False)
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 27, 'output_tokens': 23, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


## Declarative Composition

**การประมวลผลแบบ Declarative**

LangChain Expression Language (LCEL) เป็น ภาษาประกาศ สำหรับการประกอบส่วนประกอบของ LangChain LangChain คอมไพล์การประมวลผล LCEL เป็น แผนการดำเนินการที่ปรับให้เหมาะสม พร้อมการประมวลผลแบบขนาน การสตรีม การติดตาม และการรองรับแบบอะซิงโครนัสโดยอัตโนมัติ

มาดูตัวอย่างเดียวกันโดยใช้ LCEL เริ่มต้นใน Python:"

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# the building blocks
template = ChatPromptTemplate.from_messages([
    ('system', 'Please answer maximize only 5 keywords on answer'),
    ('human', '{question}'),
])
model = ChatOpenAI(openai_api_key=userdata.get('OPENAI_API_KEY'))
# combine them with the | operator

chatbot = template | model
# use it
result = chatbot.invoke({
    "question": "Question: Synonyme of Hi"
})

In [ ]:
pprint(result.content)

'1. Hello\n2. Hey\n3. Greetings\n4. Howdy\n5. Hi there'


ที่สำคัญคือ บรรทัดสุดท้ายเหมือนกันในทั้งสองตัวอย่าง นั่นคือ คุณใช้ฟังก์ชันและลำดับ LCEL ในลักษณะเดียวกัน โดยใช้ `invoke/stream/batch` และในเวอร์ชันนี้ คุณไม่จำเป็นต้องทำอะไรเพิ่มเติมเพื่อใช้การสตรีม เริ่มต้นใน Python:

In [ ]:
chatbot = template | model
for part in chatbot.stream({
    "question": "Please answer maximize only 5 keywords on answer. Question: Synonyme of Hi"
}):
    print(part)

content='' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content='Hello' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content=',' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content=' hey' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content=',' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content=' greetings' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content=',' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content=' hi' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content='ya' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8a17-95f19346b41b'
content=',' additional_kwargs={} response_metadata={} id='run-e399f125-2360-40f5-8

In [ ]:
chatbot = template | model
result = await chatbot.ainvoke({
    "question": "Please answer maximize only 5 keywords on answer. Question: Synonyme of Hi"
})
pprint(result.content)

'Hello, Greetings, Hey, Howdy, Hiya'


# Summary

**สรุป**

ในบทนี้ คุณได้เรียนรู้เกี่ยวกับส่วนประกอบพื้นฐานและองค์ประกอบสำคัญที่จำเป็นสำหรับการสร้างแอปพลิเคชัน LLM โดยใช้ LangChain แอปพลิเคชัน LLM โดยพื้นฐานแล้วเป็นห่วงโซ่ที่ประกอบด้วยโมเดลภาษาขนาดใหญ่เพื่อทำการคาดการณ์ คำแนะนำพร้อมท์เพื่อนำทางโมเดลไปสู่เอาต์พุตที่ต้องการ และตัวแยกวิเคราะห์ผลลัพธ์ (optional) เพื่อแปลงรูปแบบของเอาต์พุตของโมเดล

ส่วนประกอบทั้งหมดของ LangChain ใช้ร่วมกันอินเทอร์เฟซเดียวกันกับเมธอด invoke, stream และ batch เพื่อจัดการอินพุตและเอาต์พุตต่างๆ พวกเขาสามารถรวมและดำเนินการแบบ Imperative ได้โดยการเรียกใช้โดยตรงหรือแบบ Declarative โดยใช้ LangChain Expression Language (LCEL)

วิธีการแบบ Imperative มีประโยชน์หากคุณตั้งใจที่จะเขียนตรรกะแบบกำหนดเองจำนวนมาก ในขณะที่วิธีการแบบ Declarative มีประโยชน์สำหรับการประกอบส่วนประกอบที่มีอยู่โดยมีการปรับแต่งจำกัด

ในบทต่อไป คุณจะได้เรียนรู้วิธีการจัดเตรียมข้อมูลภายนอกให้กับแชทบอท AI ของคุณในฐานะ บริบท เพื่อให้คุณสามารถสร้างแอปพลิเคชัน LLM ที่ช่วยให้คุณ "แชท" กับข้อมูลของคุณได้